In [1]:
from data.dataloaders import create_dataloaders

train_dataloader, val_dataloader = create_dataloaders(
    imgs_dir= "../BT_dataset/images/",
    masks_dir= "../BT_dataset/masks/",
    batch_size=2,
)

Train dataset size: 2451
Validation dataset size: 613


In [ ]:
from model import TumorSegmentaionModel
from training.losses import CombinedLoss
from monai.metrics import DiceMetric, HausdorffDistanceMetric

from training.training_loop import validate

model = TumorSegmentaionModel()

model = model.to("cuda") 
    
loss_fn = CombinedLoss() 
metric = {
        # 2.1. Dice Metric: Measures the overlap between prediction and ground truth.
        # It is the primary metric for most segmentation tasks.
        'dice': DiceMetric(
            include_background=False, # Crucial: only evaluate the tumor class, not the background.
            reduction='mean',
            get_not_nans=False
        ),
        
        # 2.2. Hausdorff Distance: Measures the distance between the boundaries of the
        # predicted and ground truth segmentations. Excellent for evaluating boundary accuracy.
        'hausdorff': HausdorffDistanceMetric(
            include_background=False,
            reduction='mean',
            percentile=95 # Use the 95th percentile to make the metric robust to outliers.
        )
    }

val_loss, dice_score, hausdorff_dist = validate(
            model = model,
            val_dataloader = val_dataloader, 
            loss_fn = loss_fn, 
            metric = metric, 
            device = "cuda")